# 3) Reconstruct target file with source collection frames

This is the final notebook of the Freesound AMPLAB session and contains the code that performs *audio mosaicing* to construct a new version of the target file by using audio frames chosen from the source collection. This notebook used the DataFrames generated in the previous notebooks which contain metadata about the Freesound sounds in the source collection, the analysis results of the source collection and the analysis results of the target audio file.

In [1]:
# Essentia
!pip install essentia

In [2]:
# Mount drive and cd to notebook folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd "/content/drive/MyDrive/Colab Notebooks/freesound-final"

import sys
COLAB_WORKDIR = "/content/drive/MyDrive/Colab Notebooks/freesound-final"
if COLAB_WORKDIR not in sys.path:
  sys.path.append(COLAB_WORKDIR)
print(sys.path)

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/freesound-final
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Colab Notebooks/freesound-final']


In [3]:
import os
import random
import pandas as pd
import essentia
import essentia.standard as estd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from IPython.display import display, Audio

In [4]:
# Load all DataFrames created in the previous notebooks
DATASET_FILENAME = 'dataframe_violin.csv'
DATAFRAME_SOURCE_UNITS_FILENAME = 'dataframe_violin_source.csv'
DATAFRAME_TARGET_FILE_FILENAME = 'dataframe_violin_target.csv'

df = pd.read_csv(open(DATASET_FILENAME), index_col=0)
df_source = pd.read_csv(open(DATAFRAME_SOURCE_UNITS_FILENAME), index_col=0)
df_target = pd.read_csv(open(DATAFRAME_TARGET_FILE_FILENAME), index_col=0)

In [11]:
# Define some util functions
# NOTE: remember that if you update these util functions and want to do a new audio mosaicing, you'll need 
# to re-run both this cell (to update the util functions) and the cell below (which uses the util functions to
# do the audio mosaicing).

loaded_audio_files = {}

def get_audio_file_segment(file_path, start_sample, n_samples):
    """Load audio file. Try to get it from memory first. If not there, open it and save in memory for next time.
    """
    if file_path not in loaded_audio_files:
        loader = estd.MonoLoader(filename=file_path)
        audio = loader()
        loaded_audio_files[file_path] = audio
    else:
        audio = loaded_audio_files[file_path]
        
    # Return segment
    return audio[start_sample:start_sample + n_samples]

def find_similar_frames(query_frame, df_source_frames, n, features):
    """Find the 'n' mosr similar frames for a given 'query_frame' from those in the given 'df_source_frames'.
    Similarity is computed using a nearest neighbours algorithm and taking only into account the feature list
    given in the 'features' parameter.
    """
    query_frame = query_frame.reshape(1,-1)
    nbrs = NearestNeighbors(n_neighbors=n, algorithm='ball_tree').fit(df_source_frames[features].values)
    distances, indices = nbrs.kneighbors(query_frame)
    return [df_source_frames.iloc[k] for k in indices[0]]

def chose_frame_from_source_collection(target_frame, df_source_frames):
    """Choose one frame from 'df_source_frames' to replace the 'target_frame'.
    This implementation chooses the source frame usinng a similarity algorithm 'find_similar_frames',
    and a specific set of similarity features for timbre (MFCC).
    You can modify this function to implement new ways to choose a frame from the source.
    
    NOTE: 'target_frame' here should have the same features as 'df_source_frames' because both the
    target file and the source collection have been analyzed with the same analysis function.
    You can list available features using 'print(list(target_frame.keys()))' and print(list(df_source_frames.keys()))
    """
    n_neighbours_to_find = 10
    # similarity_features = ['mean_pitch', 'mfcc_0', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12']
    similarity_features = ['mean_pitch']

    # Find the 10 most similar frames to the target_frame from df_source_framesdf_source_units 
    query_frame = target_frame[similarity_features].values
    similar_frames = find_similar_frames(query_frame, df_source_frames, n_neighbours_to_find, similarity_features)

    # Choose randomly among the top n frames
    return random.choice(similar_frames)

    # Choose the first one as is the most similar
    # return similar_frames[0]
    

In [12]:
# Do the reconstruction (audio mosaicing) of the target file using audio chunks (units, frames) from the sounds in the source collection

# Load target audio file to get its total length and to use it later
target_sound_filename = df_target.iloc[0]['path']
target_audio = estd.MonoLoader(filename=target_sound_filename)()
total_length_target_audio = len(target_audio)
print('total_length_target_audio', total_length_target_audio)

# Init array where to put the audio of the reconstructed file
generated_audio = np.zeros(total_length_target_audio)

# Init list where to store IDs of sounds used in the reconstruction
selected_freesound_ids = []

# Iterate over the analyzed frame of the target file
print('Reconstructing audio file...')
for i in range(0, len(df_target)):
    try:
      target_frame = df_target.iloc[i]  # Get current frame
      
      # Choose one frame from the source collection to replace the target frame
      most_similar_frame = chose_frame_from_source_collection(target_frame, df_source)
      
      # Store freesound ID of the original sound where the 'most_similar_frame' belongs to
      selected_freesound_ids.append(most_similar_frame['freesound_id'])
      
      # Get the audio segment corresponding to the 'most_similar_frame'
      target_frame_n_samples = target_frame['end_sample'] - target_frame['start_sample']
      most_similar_frame_audio = get_audio_file_segment(most_similar_frame['path'], most_similar_frame['start_sample'], target_frame_n_samples)

      # Add audio segment to the reconstructed audio array
      generated_audio[target_frame['start_sample']:target_frame['start_sample']+len(most_similar_frame_audio)] = most_similar_frame_audio
    except:
      pass

# Store the results in a WAV file
generated_audio_filename = '{0}.reconstructed.wav'.format(target_sound_filename)
estd.MonoWriter(filename=generated_audio_filename, format='wav', sampleRate=44100)(essentia.array(generated_audio))
print('Audio generated and saved in {0}!\nIt contains audio from the following sounds:'.format(generated_audio_filename))
display(df.loc[df['freesound_id'].isin(selected_freesound_ids)])  # Show metadata for the Freesound sounds used in the reconstruction


total_length_target_audio 441000
Reconstructing audio file...
Audio generated and saved in targets/short_V1bFr2SWP1I.wav.reconstructed.wav!
It contains audio from the following sounds:


,name,username,license,tags,freesound_id,path
34,violin harmony 5,PhonosUPF,http://creativecommons.org/publicdomain/zero/1.0/,"['violin', 'transformation', 'harmony']",494679,files/494679_10350281-hq.ogg
35,violin sequence 9,PhonosUPF,http://creativecommons.org/publicdomain/zero/1.0/,"['violin', 'glissando', 'transformation']",494541,files/494541_10350281-hq.ogg
63,Bluesy Violin repeating double stops.wav,TheFlyFishingFilmmaker,http://creativecommons.org/licenses/by/3.0/,"['lick', 'violins', 'jazz', 'arpeggio', 'fiddl...",621598,files/621598_6501596-hq.ogg
78,vi.wav,The-Sacha-Rush,http://creativecommons.org/publicdomain/zero/1.0/,"['archestra', 'instrument', 'loop', 'melodic',...",71409,files/71409_685248-hq.ogg


In [13]:
# Show further results of the reconstruction

# Plot waveforms
plt.figure(figsize=(15,5))
plt.plot(target_audio)
plt.axis([0, len(target_audio), -1, 1])
plt.title('Target audio')
plt.show()

plt.figure(figsize=(15,5))
plt.plot(generated_audio)
plt.axis([0, len(target_audio), -1, 1])
plt.title('Reconstructed')
plt.show()

# Show audio players
print('Target audio')
display(Audio(target_audio, rate=44100))

print('Reconstructed')
display(Audio(generated_audio, rate=44100))

print('Mix of both signals')
display(Audio(generated_audio * 0.5 + target_audio * 0.25, rate=44100))

Output hidden; open in https://colab.research.google.com to view.

In [14]:
estd.AudioWriter(filename='over_the_rainbow_violin_mix.mp3', format='mp3')(estd.StereoMuxer()(generated_audio * 0.5, target_audio * 0.1))